## Getting Started

### Prerequisites
1. Create an Ollama account at [ollama.com](https://ollama.com)
2. Generate an API key from [ollama.com/settings/keys](https://ollama.com/settings/keys)
3. Set the API key in the `.env` file in this folder

### Installation
```bash
pip install ollama python-dotenv requests
```

In [1]:
# Setup: Install required packages and import libraries
import sys
import subprocess

packages = ['ollama', 'python-dotenv', 'requests']

for package in packages:
    try:
        __import__(package.replace('-', '_'))
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

print("✓ All packages installed successfully!")

Installing python-dotenv...
✓ All packages installed successfully!


In [ ]:
# Import all required libraries
import os
from pathlib import Path
from dotenv import load_dotenv
import ollama
from typing import Dict, Any

# Load environment variables from .env file
# Use Path.cwd() since __file__ is not defined in Jupyter notebooks
env_path = Path.cwd() / '.env'
if env_path.exists():
    load_dotenv(env_path)
else:
    # Try to find .env in the ollama-cloud directory
    load_dotenv(dotenv_path='d:\\ollama-n8n\\ollama-cloud\\.env')

# Get API key and configuration
OLLAMA_API_KEY = os.getenv('OLLAMA_API_KEY')
OLLAMA_ENDPOINT = os.getenv('OLLAMA_API_ENDPOINT', 'https://ollama.com')
DEFAULT_MODEL = os.getenv('DEFAULT_MODEL', 'gpt-oss:120b')

# Verify API key is set
if not OLLAMA_API_KEY or OLLAMA_API_KEY == 'your_api_key_here':
    print("WARNING: OLLAMA_API_KEY not set in .env file!")
    print("Please set your API key in the .env file: https://ollama.com/settings/keys")
else:
    print(f"✓ Ollama API configured")
    print(f"  Endpoint: {OLLAMA_ENDPOINT}")
    print(f"  Default Model: {DEFAULT_MODEL}")

# Initialize Ollama client for cloud API
client = ollama.Client(
    host=OLLAMA_ENDPOINT,
    headers={'Authorization': f'Bearer {OLLAMA_API_KEY}'}
)

✓ Ollama API configured
  Endpoint: https://ollama.com
  Default Model: gpt-oss:120b


## Tool Calling Capability - Extend Models with Functions

Tool calling allows models to request execution of custom functions. The model decides when and how to use tools based on the user's request.

**Use Cases:**
- Mathematical calculations
- Database queries
- API calls
- Real-time data retrieval
- Complex workflows

In [3]:
def test_tool_calling(model: str = 'gpt-oss:120b', user_query: str = "What is 17 × 23? Also, what is the square root of 529?") -> Dict[str, Any]:
    """
    Test tool calling - models can request tool execution.
    
    Args:
        model: Model with tool calling support
        user_query: Question that requires tool use
    
    Returns:
        Conversation with tool calls and results
    """
    
    # Define available tools
    def multiply(a: float, b: float) -> float:
        """Multiply two numbers"""
        return a * b
    
    def square_root(n: float) -> float:
        """Calculate square root"""
        import math
        return math.sqrt(n)
    
    def get_weather(location: str) -> str:
        """Get weather for a location (mock)"""
        return f"Weather in {location}: Sunny, 22°C"
    
    # Define tools for the model
    tools = [
        {
            "type": "function",
            "function": {
                "name": "multiply",
                "description": "Multiply two numbers",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "a": {"type": "number", "description": "First number"},
                        "b": {"type": "number", "description": "Second number"}
                    },
                    "required": ["a", "b"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "square_root",
                "description": "Calculate the square root of a number",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "n": {"type": "number", "description": "The number to get square root of"}
                    },
                    "required": ["n"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_weather",
                "description": "Get current weather for a location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {"type": "string", "description": "City name"}
                    },
                    "required": ["location"]
                }
            }
        }
    ]
    
    print(f"   Testing tool calling with {model}...")
    print(f"   Query: {user_query}\n")
    
    # Map function names to actual functions
    available_functions = {
        'multiply': multiply,
        'square_root': square_root,
        'get_weather': get_weather
    }
    
    messages = [{'role': 'user', 'content': user_query}]
    conversation_history = []
    
    # Initial request with tools
    response = client.chat(
        model=model,
        messages=messages,
        tools=tools,
        stream=False
    )
    
    print(f"   Model's Initial Response:")
    print(f"   Content: {response.message.content}")
    
    # Check for tool calls
    if hasattr(response.message, 'tool_calls') and response.message.tool_calls:
        print(f"\n Tool Calls Requested: {len(response.message.tool_calls)}")
        
        # Process each tool call
        for tool_call in response.message.tool_calls:
            func_name = tool_call.function.name
            func_args = tool_call.function.arguments
            
            print(f"\n   → Calling: {func_name}({func_args})")
            
            # Execute the function
            if func_name in available_functions:
                result = available_functions[func_name](**func_args)
                print(f"     Result: {result}")
                
                # Add tool result to conversation
                messages.append(response.message)
                messages.append({
                    'role': 'tool',
                    'content': str(result),
                    'tool_name': func_name
                })
    
    # Get final response if tools were used
    if len(messages) > 1:
        final_response = client.chat(
            model=model,
            messages=messages,
            stream=False
        )
        print(f"\n✓ Final Answer:")
        print(f"   {final_response.message.content}\n")
        return {
            'initial_response': response.message.content,
            'tool_calls': [tc.function.name for tc in (response.message.tool_calls or [])],
            'final_response': final_response.message.content
        }
    
    return {'response': response.message.content, 'tool_calls': []}

# Test tool calling
tool_result = test_tool_calling()

   Testing tool calling with gpt-oss:120b...
   Query: What is 17 × 23? Also, what is the square root of 529?

   Model's Initial Response:
   Content: 

 Tool Calls Requested: 1

   → Calling: multiply({'a': 17, 'b': 23})
     Result: 391

✓ Final Answer:
   

